# Gesture Recognition
In this group project, you are going to build a 2D CNN + RNN model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#we will use cv2 here
import cv2

import datetime
import os
import matplotlib.pyplot as plt
from keras.applications.resnet import ResNet50

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, LSTM, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2

We set the random seed so that the results don't vary drastically.

In [8]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [11]:
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
print(len(train_doc), len(val_doc))
batch_size = 50 #experiment with the batch size

663 100


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [12]:
scale_f = 70

### We have modified the generator function by adding 2 more parameters scale_f -> size of image to be rescaled and n_i -> number of images to train on

In [13]:
def generator(source_path, folder_list, batch_size, scale_f, n_i):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #create a list of image numbers you want to use for a particular video
    img_idx = [i for i in range(30)]
    if(n_i == 20):
        img_idx = [0,1,4,5,7,8,10,11,13,14,16,17,19,20,22,23,25,26,28,29]
    elif(n_i == 15):
        img_idx = [i for i in range(0,30,2)]
    while True:
        t = np.random.permutation(folder_list)
        # calculate the number of batches
        num_batches = int(len(folder_list)/batch_size)
    
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),scale_f,scale_f,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    #We have 160*120 and 360*360 images, we will resize every image to scale_f*scale_f
                    image = cv2.resize(image, (scale_f, scale_f))
                    
                    
                    #normalising using min-max method
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - np.min(image[:,:,0])/np.max(image[:,:,0]) - np.min(image[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - np.min(image[:,:,1])/np.max(image[:,:,1]) - np.min(image[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - np.min(image[:,:,2])/np.max(image[:,:,2]) - np.min(image[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        
        left_samples = len(folder_list)%batch_size
        offset = len(folder_list) - left_samples
        if (left_samples > 0):
            batch_data = np.zeros((left_samples,len(img_idx),scale_f,scale_f,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((left_samples,5)) # batch_labels is the one hot representation of the output
            for folder in range(left_samples): # iterate over the left_samples
                imgs = os.listdir(source_path+'/'+ t[folder + offset].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + offset].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    #We have 160*120 and 360*360 images, we will resize every image to scale_f*scale_f
                    image = cv2.resize(image, (scale_f, scale_f))


                    #normalising using min-max method
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - np.min(image[:,:,0])/np.max(image[:,:,0]) - np.min(image[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - np.min(image[:,:,1])/np.max(image[:,:,1]) - np.min(image[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - np.min(image[:,:,2])/np.max(image[:,:,2]) - np.min(image[:,:,2])#normalise and feed in the image

                batch_labels[folder, int(t[folder + offset].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [14]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 1# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 1


# Model 1
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

### Build the Model

In [16]:
tf.random.set_seed(30)
#initially building a simple model to test whether model is fitting and generator is running or not
model = Sequential()
n_i = 30
model.add(TimeDistributed(Conv2D(64,(3,3),activation='relu',input_shape=(n_i, scale_f, scale_f, 3))))#(inputs))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

# flatten
model.add(TimeDistributed(Flatten()))

#addig GRU layers
model.add(GRU(64))
model.add(Dropout(0.25))
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [17]:
optimiser = 'sgd'#optimizer

#building model with input
input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 30, 68, 68, 64)   1792      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 30, 34, 34, 64)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 30, 34, 34, 64)   0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 30, 73984)        0         
 tributed)                                                       
                                                                 
 gru (GRU)                   (None, 64)                1

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [18]:
train_generator = generator(train_path, train_doc, batch_size, scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size, scale_f, n_i)

In [19]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001) # Reducelronplateau code
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [20]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

### Compile and Fit the Model

In [21]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
history=model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  train ; batch size = 50
Epoch 1/10
14/14 [==============================] - ETA: 0s - loss: 1.7681 - categorical_accuracy: 0.2112Source path =  val ; batch size = 50

Epoch 1: saving model to model_init_2022-05-16--17_28_33.805466\model-00001-1.76814-0.21116-1.65975-0.16000.h5
14/14 [==============================] - 79s 5s/step - loss: 1.7681 - categorical_accuracy: 0.2112 - val_loss: 1.6597 - val_categorical_accuracy: 0.1600 - lr: 0.0100
Epoch 2/10
14/14 [==============================] - ETA: 0s - loss: 1.6820 - categorical_accuracy: 0.1885
Epoch 2: saving model to model_init_2022-05-16--17_28_33.805466\model-00002-1.68203-0.18854-1.65610-0.14000.h5
14/14 [==============================] - 80s 6s/step - loss: 1.6820 - categorical_accuracy: 0.1885 - val_loss: 1.6561 - val_categorical_accuracy: 0.1400 - lr: 0.0100
Epoch 3/10
14/14 [==============================] - ETA: 0s - loss: 1.6529 - categorical_accuracy: 0.1900
Epoch 3: saving model to model_init_2022-05-16--17_2

## Findings: Model 1
* Model has very low accuracy.
* We will create new model with additional conv2d timedistributed layers
- Train loss: 1.6495 
- Train categorical_accuracy: 0.1825 
- val_loss: 1.6119 
- val_categorical_accuracy: 0.2000

# Model 2
* We have included additional conv2d timedistributed layers

### Build the Model

In [22]:
#since Last model is very bad in terms of accuracy, we will add more conv2d timedistributed layers 

model = Sequential()
n_i = 30
model.add(TimeDistributed(Conv2D(64,(3,3),activation='relu',input_shape=(n_i, scale_f, scale_f, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(32, (3, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same')))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(64, (3, 3))))
model.add(TimeDistributed((Activation('relu'))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))


# flatten
model.add(TimeDistributed(Flatten()))

#adding GRU layer with dropout
model.add(GRU(64,return_sequences=True))
model.add(TimeDistributed(Dropout(0.25)))

#flattening again for dense input
model.add(Flatten())

#taking a single dense before output softmax
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax'))

optimiser = 'sgd'#optimizer

input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_4 (TimeDis  (None, 30, 68, 68, 64)   1792      
 tributed)                                                       
                                                                 
 time_distributed_5 (TimeDis  (None, 30, 68, 68, 64)   0         
 tributed)                                                       
                                                                 
 time_distributed_6 (TimeDis  (None, 30, 68, 68, 64)   256       
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 30, 66, 66, 32)   18464     
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 30, 66, 66, 32)  

In [23]:
#changing batch size to fit in memory
batch_size = 30

#do for 10 epochs
num_epochs = 10
train_generator = generator(train_path, train_doc, batch_size,scale_f,n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f,n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001) # Reducelronplateau code
callbacks_list = [checkpoint, LR]





In [24]:
#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

### Compile and Fit the Model

In [25]:
    
#fit model
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

C:\Users\44775\AppData\Local\Temp\ipykernel_13840\3178268726.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  train ; batch size = 30
Epoch 1/10
23/23 [==============================] - ETA: 0s - loss: 1.5230 - categorical_accuracy: 0.3605Source path =  val ; batch size = 30

Epoch 1: saving model to model_init_2022-05-16--17_28_33.805466\model-00001-1.52304-0.36048-2.63750-0.18000.h5
23/23 [==============================] - 213s 9s/step - loss: 1.5230 - categorical_accuracy: 0.3605 - val_loss: 2.6375 - val_categorical_accuracy: 0.1800 - lr: 0.0100
Epoch 2/10
23/23 [==============================] - ETA: 0s - loss: 1.2686 - categorical_accuracy: 0.5189
Epoch 2: saving model to model_init_2022-05-16--17_28_33.805466\model-00002-1.26863-0.51885-4.27341-0.26000.h5
23/23 [==============================] - 207s 9s/step - loss: 1.2686 - categorical_accuracy: 0.5189 - val_loss: 4.2734 - val_categorical_accuracy: 0.2600 - lr: 0.0100
Epoch 3/10
23/23 [==============================] - ETA: 0s - loss: 0.9509 - categorical_accuracy: 0.6244
Epoch 3: saving model to model_init_2022-05-16--17

## Findings: Model 2

* Model 2 is overfitting. Accuracy on train dataset is 0.9020 where as Accuracy on validation dataset is 0.7500
* We will experiment by increasing the size of scaled images and check if the new model can manage the issue of overfitting.
* 
  Total params: 3,046,653
  Trainable params: 3,046,205
  Non-trainable params: 448
  
- Train loss: 0.2903 
- Train categorical_accuracy: 0.9020 
- val_loss: 0.6828 
- val_categorical_accuracy: 0.7500 

# Model 3
* Creating a model with size of the scaled image =70
* adding another set of conv2D timedistributed layer for better feature extraction

### Build the Model

In [26]:

#size of scaled image =70
scale_f = 70

model = Sequential()
n_i = 30
model.add(TimeDistributed(Conv2D(64,(3,3),activation='relu',input_shape=(n_i, scale_f, scale_f, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(32, (3, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same')))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(64, (3, 3))))
model.add(TimeDistributed((Activation('relu'))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))


#we have added another set of conv2d timedistributed layers for better feature extraction
model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same')))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(128, (3, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

# flatten
model.add(TimeDistributed(Flatten()))

#adding GRU layers
model.add(GRU(64,return_sequences=True))


model.add(Flatten())

#increasing the dense layer size to 512 for better learning
model.add(Dense(512,kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

optimiser = 'sgd'#optimizer

input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_22 (TimeDi  (None, 30, 68, 68, 64)   1792      
 stributed)                                                      
                                                                 
 time_distributed_23 (TimeDi  (None, 30, 68, 68, 64)   0         
 stributed)                                                      
                                                                 
 time_distributed_24 (TimeDi  (None, 30, 68, 68, 64)   256       
 stributed)                                                      
                                                                 
 time_distributed_25 (TimeDi  (None, 30, 66, 66, 32)   18464     
 stributed)                                                      
                                                                 
 time_distributed_26 (TimeDi  (None, 30, 66, 66, 32)  

### Compile and Fit the Model

In [27]:
batch_size = 20
num_epochs = 10
train_generator = generator(train_path, train_doc, batch_size,scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f, n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001) #Reducelronplateau code 
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  train ; batch size = 20


C:\Users\44775\AppData\Local\Temp\ipykernel_13840\1776157345.py:34: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/10
34/34 [==============================] - ETA: 0s - loss: 9.4205 - categorical_accuracy: 0.4072Source path =  val ; batch size = 20

Epoch 1: saving model to model_init_2022-05-16--17_28_33.805466\model-00001-9.42045-0.40724-12.43602-0.23000.h5
34/34 [==============================] - 262s 8s/step - loss: 9.4205 - categorical_accuracy: 0.4072 - val_loss: 12.4360 - val_categorical_accuracy: 0.2300 - lr: 0.0100
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 9.0109 - categorical_accuracy: 0.5656
Epoch 2: saving model to model_init_2022-05-16--17_28_33.805466\model-00002-9.01089-0.56561-11.42424-0.23000.h5
34/34 [==============================] - 229s 7s/step - loss: 9.0109 - categorical_accuracy: 0.5656 - val_loss: 11.4242 - val_categorical_accuracy: 0.2300 - lr: 0.0100
Epoch 3/10
34/34 [==============================] - ETA: 0s - loss: 8.5514 - categorical_accuracy: 0.7436
Epoch 3: saving model to model_init_2022-05-16--17_28_33.805466\model-00003-8.55141-0

### Findings
* models in few epochs has manage to sort the overfitting issue but later epoch still shown overfitting. 
* we will try to reduce the overfitting as well as loss in next model

- Train loss: 7.3055 
- Train categorical_accuracy: 0.9367 
- val_loss: 7.7596 
- val_categorical_accuracy: 0.7600

* Total params: 2,182,565
  Trainable params: 2,181,605
  Non-trainable params: 960
___________________________

# Model 4

### Build, Compile and Fit the Model

In [28]:
#now let's again check with decreasing learning rate at each epoch (patience = 1)
batch_size = 20
num_epochs = 20
train_generator = generator(train_path, train_doc, batch_size,scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f, n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001) #Reducelronplateau code
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  train ; batch size = 20


C:\Users\44775\AppData\Local\Temp\ipykernel_13840\2170958243.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 7.1317 - categorical_accuracy: 0.9608Source path =  val ; batch size = 20

Epoch 1: saving model to model_init_2022-05-16--17_28_33.805466\model-00001-7.13168-0.96078-7.74347-0.76000.h5
34/34 [==============================] - 233s 7s/step - loss: 7.1317 - categorical_accuracy: 0.9608 - val_loss: 7.7435 - val_categorical_accuracy: 0.7600 - lr: 0.0100
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 6.9974 - categorical_accuracy: 0.9819
Epoch 2: saving model to model_init_2022-05-16--17_28_33.805466\model-00002-6.99736-0.98190-7.61347-0.78000.h5
34/34 [==============================] - 225s 7s/step - loss: 6.9974 - categorical_accuracy: 0.9819 - val_loss: 7.6135 - val_categorical_accuracy: 0.7800 - lr: 0.0100
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 6.8795 - categorical_accuracy: 0.9940
Epoch 3: saving model to model_init_2022-05-16--17_28_33.805466\model-00003-6.87951-0.993

### Findings
* Model is still overfitting.So we will use transfer learning in the next model
- Train loss: 6.6168 
- Train categorical_accuracy: 0.9985 
- val_loss: 7.0530
- val_categorical_accuracy: 0.8300 

# Model 5
 * We should use Resnet50 for transfer learning
 * We kept the learning rate at 0.0004

### Build the Model

In [31]:

#let's try transfer learning with Resnet
scale_f = 197  #size of image limitation with Resnet is 197
n_i = 20

# create a Sequential model
model = Sequential()


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(scale_f, scale_f, 3))

#making some layers trainable
split_at = 150
for layer in base_model.layers[:split_at]: layer.trainable = False
for layer in base_model.layers[split_at:]: layer.trainable = True

model.add(TimeDistributed(base_model, input_shape=(n_i, scale_f, scale_f, 3)))
model.add(TimeDistributed(GlobalAveragePooling2D()))
# flatten
model.add(TimeDistributed(Flatten()))

#adding GRU/LSTM layers
model.add(GRU(1000,return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dropout(0.30)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(5, activation='softmax'))

optimiser = 'sgd' #optimizer

input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_50 (TimeDi  (None, 20, 7, 7, 2048)   23587712  
 stributed)                                                      
                                                                 
 time_distributed_51 (TimeDi  (None, 20, 2048)         0         
 stributed)                                                      
                                                                 
 time_distributed_52 (TimeDi  (None, 20, 2048)         0         
 stributed)                                                      
                                                                 
 gru_4 (GRU)                 (None, 20, 1000)          9150000   
                                                                 
 time_distributed_53 (TimeDi  (None, 20, 100)          100100    
 stributed)                                           

### Compile and Fit the Model

In [32]:
#increasing epochs and decreasing learning rate to 0.0004

batch_size = 20
num_epochs = 15
train_generator = generator(train_path, train_doc, batch_size,scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f, n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.0004) #Reducelronplateau code
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate when we are fine tuning the initial layers
sgd = tf.keras.optimizers.SGD(lr=0.0004, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  train ; batch size = 20


C:\Users\44775\AppData\Local\Temp\ipykernel_13840\1976619657.py:40: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/15
34/34 [==============================] - ETA: 0s - loss: 1.6212 - categorical_accuracy: 0.2293 Source path =  val ; batch size = 20

Epoch 1: saving model to model_init_2022-05-16--17_28_33.805466\model-00001-1.62121-0.22926-1.51477-0.32000.h5
34/34 [==============================] - 1043s 31s/step - loss: 1.6212 - categorical_accuracy: 0.2293 - val_loss: 1.5148 - val_categorical_accuracy: 0.3200 - lr: 4.0000e-04
Epoch 2/15
34/34 [==============================] - ETA: 0s - loss: 1.4237 - categorical_accuracy: 0.4072 
Epoch 2: saving model to model_init_2022-05-16--17_28_33.805466\model-00002-1.42372-0.40724-1.19059-0.65000.h5
34/34 [==============================] - 1032s 30s/step - loss: 1.4237 - categorical_accuracy: 0.4072 - val_loss: 1.1906 - val_categorical_accuracy: 0.6500 - lr: 4.0000e-04
Epoch 3/15
34/34 [==============================] - ETA: 0s - loss: 1.0678 - categorical_accuracy: 0.6139 
Epoch 3: saving model to model_init_2022-05-16--17_28_33.805466\model-0000

### Findings
* Resnet50 model has managed the overfitting well. Transfer learning has been successful.
- Train loss: 0.0312 
- Train categorical_accuracy: 0.9970 
- val_loss: 0.0513 
- val_categorical_accuracy: 0.9900 

# Model 6

 * now we should fine tune this model more,  train over 140th layer
 * we also changing GRU layer to LSTM
 * We kept the learning rate at 0.0005

### Build the model

In [33]:

scale_f = 197 #size of image limitation with Resnet
n_i = 20

# create a Sequential model
model = Sequential()

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(scale_f, scale_f, 3))

#making some layers trainable
split_at = 140
for layer in base_model.layers[:split_at]: layer.trainable = False
for layer in base_model.layers[split_at:]: layer.trainable = True

model.add(TimeDistributed(base_model, input_shape=(n_i, scale_f, scale_f, 3)))
model.add(TimeDistributed(GlobalAveragePooling2D()))
# flatten
model.add(TimeDistributed(Flatten()))

#adding GRU/LSTM layers of more units and adding timedistributed dense layer of 128 neurons each
model.add(LSTM(1024,return_sequences=True))
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(TimeDistributed(Dropout(0.30)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(5, activation='softmax'))

optimiser = 'sgd' #optimizer

input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_55 (TimeDi  (None, 20, 7, 7, 2048)   23587712  
 stributed)                                                      
                                                                 
 time_distributed_56 (TimeDi  (None, 20, 2048)         0         
 stributed)                                                      
                                                                 
 time_distributed_57 (TimeDi  (None, 20, 2048)         0         
 stributed)                                                      
                                                                 
 lstm (LSTM)                 (None, 20, 1024)          12587008  
                                                                 
 time_distributed_58 (TimeDi  (None, 20, 128)          131200    
 stributed)                                           

### Compile and Fit the Model

In [34]:
#increasing epochs to 25 and making learning rate to 0.0005

batch_size = 20
num_epochs = 25
train_generator = generator(train_path, train_doc, batch_size,scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f, n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.0005) #Reducelronplateau code
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
# Choosing learning rate of 0.0005

sgd = tf.keras.optimizers.SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  train ; batch size = 20


C:\Users\44775\AppData\Local\Temp\ipykernel_13840\1777622456.py:40: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/25
34/34 [==============================] - ETA: 0s - loss: 1.5662 - categorical_accuracy: 0.2821 Source path =  val ; batch size = 20

Epoch 1: saving model to model_init_2022-05-16--17_28_33.805466\model-00001-1.56621-0.28205-1.42013-0.48000.h5
34/34 [==============================] - 1145s 34s/step - loss: 1.5662 - categorical_accuracy: 0.2821 - val_loss: 1.4201 - val_categorical_accuracy: 0.4800 - lr: 5.0000e-04
Epoch 2/25
34/34 [==============================] - ETA: 0s - loss: 1.1777 - categorical_accuracy: 0.6018 
Epoch 2: saving model to model_init_2022-05-16--17_28_33.805466\model-00002-1.17772-0.60181-0.94130-0.73000.h5
34/34 [==============================] - 1148s 34s/step - loss: 1.1777 - categorical_accuracy: 0.6018 - val_loss: 0.9413 - val_categorical_accuracy: 0.7300 - lr: 5.0000e-04
Epoch 3/25
34/34 [==============================] - ETA: 0s - loss: 0.7295 - categorical_accuracy: 0.7526 
Epoch 3: saving model to model_init_2022-05-16--17_28_33.805466\model-0000

Epoch 22/25
34/34 [==============================] - ETA: 0s - loss: 0.0080 - categorical_accuracy: 1.0000 
Epoch 22: saving model to model_init_2022-05-16--17_28_33.805466\model-00022-0.00799-1.00000-0.04621-0.99000.h5
34/34 [==============================] - 1059s 31s/step - loss: 0.0080 - categorical_accuracy: 1.0000 - val_loss: 0.0462 - val_categorical_accuracy: 0.9900 - lr: 5.0000e-04
Epoch 23/25
34/34 [==============================] - ETA: 0s - loss: 0.0104 - categorical_accuracy: 0.9970 
Epoch 23: saving model to model_init_2022-05-16--17_28_33.805466\model-00023-0.01037-0.99698-0.04019-0.99000.h5
34/34 [==============================] - 1025s 30s/step - loss: 0.0104 - categorical_accuracy: 0.9970 - val_loss: 0.0402 - val_categorical_accuracy: 0.9900 - lr: 5.0000e-04
Epoch 24/25
34/34 [==============================] - ETA: 0s - loss: 0.0061 - categorical_accuracy: 1.0000 
Epoch 24: saving model to model_init_2022-05-16--17_28_33.805466\model-00024-0.00609-1.00000-0.05215-0.980

### Findings:
 * Perfect accuracy on train and valid data. Even Loss has reduced.
- Train loss: 0.0050 
- Train categorical_accuracy: 1.0000 
- val_loss: 0.0088 
- val_categorical_accuracy: 1.0000 
 *        
    Total params: 36,470,149
    Trainable params: 27,860,485
    Non-trainable params: 8,609,664